In [25]:
import pandas as pd
dataSet= pd.read_csv("biodegradable_a.csv", sep=",")
dataSet

,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,NaN,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,NaN,2.0,...,0.0,0.0,1.0,NaN,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,NaN,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,NaN,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,NaN,2.0,3.0,...,0.0,0.0,0.0,NaN,3.059198,0.0,7.908802,0.0,0.0,RB


### Remover os espaços em branco do DataSet:

##### KNNInputer

In [26]:
import pandas as pd
from sklearn.impute import KNNImputer

# Escolhemos 5 vizinhos de modo a obter uma valor médio para cada valor em falta 
# ou seja nem muito adaptado aos dados (overfitting) nem muito suave (underfitting)
imputer = KNNImputer(n_neighbors=5, weights="uniform");
badColumns = dataSet.columns[dataSet.isnull().any()];

print(badColumns)

dataSet[badColumns] = imputer.fit_transform(dataSet[badColumns]);

dataSet

Index(['F01', 'C', 'nCp', 'HyWi_B', 'F03_CO', 'Me', 'nCIR', 'SpMax_A', 'SdO',
       'nCrt', 'SpMax_B', 'Psi_i_A', 'nX'],
      dtype='object')


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,RB
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,RB
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,RB
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,RB
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,RB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,RB
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,RB
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,RB
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,RB


Categorizar a variável Biodegradable em binário.

In [27]:
binario = pd.get_dummies(dataSet, columns=['Biodegradable'], prefix=['Biodegradable']);

dataSet = pd.concat([dataSet, binario['Biodegradable_RB']], axis=1)
dataSet = dataSet.drop('Biodegradable', axis=1)
dataSet['Biodegradable_RB'] = dataSet['Biodegradable_RB'].astype(int)


dataSet


,SpMax_L,J_Dz(e),nHM,F01,F04,NssssC,nCb,C,nCp,nO,...,C_026,F02_CN,nHDon,SpMax_B,Psi_i_A,nN,SM6_B,nArCOOR,nX,Biodegradable_RB
0,3.919000,2.690900,0.0,0.0,0.0,0.0,0.0,31.400000,2.0,0.0,...,0.0,0.0,0.0,2.949000,1.591000,0.0,7.253000,0.0,0.0,1
1,4.170000,2.114400,0.0,0.0,0.0,0.0,0.0,30.800000,1.0,1.0,...,0.0,0.0,0.0,3.315000,1.967000,0.0,7.257000,0.0,0.0,1
2,3.932000,3.251200,0.0,0.0,0.0,0.0,0.0,26.700000,2.0,4.0,...,0.0,0.0,1.0,3.183760,2.417000,0.0,7.601000,0.0,0.0,1
3,3.000000,2.709800,0.0,0.0,0.0,0.0,0.0,20.000000,1.0,2.0,...,0.0,0.0,1.0,3.091871,5.000000,0.0,6.690000,0.0,0.0,1
4,4.236000,3.394400,0.0,0.0,0.0,0.0,0.0,29.400000,2.0,4.0,...,0.0,0.0,0.0,3.351000,2.405000,0.0,8.003000,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4559,4.175650,3.454649,0.0,0.0,0.0,0.0,0.0,27.300000,2.0,2.0,...,0.0,0.0,0.0,3.398382,3.241090,0.0,7.406748,0.0,0.0,1
4560,3.853731,2.747142,0.0,0.0,0.0,0.0,0.0,33.300000,0.0,0.0,...,0.0,0.0,1.0,3.429148,2.719309,0.0,7.511511,0.0,0.0,1
4561,4.294771,3.471226,0.0,0.0,0.0,0.0,0.0,28.422154,2.0,2.0,...,0.0,0.0,0.0,3.351666,2.729456,0.0,7.699324,0.0,0.0,1
4562,4.560376,3.896390,0.0,0.0,0.0,0.0,0.0,31.872380,2.0,3.0,...,0.0,0.0,0.0,3.509342,3.059198,0.0,7.908802,0.0,0.0,1


### Scaling data... To see wich is the best scaler



In [28]:
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef, confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler,StandardScaler

# Método retirado do notebook: EC_TP04.ipynb -> 2022/2023
# Este método imprime um conjunto de estatísticas sobre um determinado modelo treinado e as suas previsoes.
def printClassResults(truth, preds):
    print("The Accuracy is: %7.4f" % accuracy_score(truth, preds))
    print("The Precision is: %7.4f" % precision_score(truth, preds))
    print("The Recall is: %7.4f" % recall_score(truth, preds))
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    #print()
    #print("This is the Confusion Matrix")
    #print(pd.DataFrame(confusion_matrix(truth, preds)))


scalers = {
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler(),
    'MinMaxScaler': MinMaxScaler()
}

def testScalers(scalers, model, results,Xtrain,Xtest,Ytrain,Ytest):

    for scaler_name, scaler in scalers.items():
        X_train_scaled = scaler.fit_transform(Xtrain)
        X_test_scaled = scaler.transform(Xtest)
        
        model.fit(X_train_scaled, Ytrain)
        
        Ypred = model.predict(X_test_scaled)
        
        accuracy = accuracy_score(Ytest, Ypred)
        
        results[scaler_name] = accuracy

        print("Scaler: ",scaler_name,"\n");
        printClassResults(Ytest,Ypred);
        print()

def scaleData(scaler,Xtrain,Xtest):
    X_train_scaled = scaler.fit_transform(Xtrain)
    X_test_scaled = scaler.transform(Xtest)
    return X_train_scaled, X_test_scaled





##### Logistic Regression:

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,LogisticRegression(max_iter=1000,random_state=0),results,X_train,X_test,y_train,y_test);




Scaler:  StandardScaler 

The Accuracy is:  0.9608
The Precision is:  0.9634
The Recall is:  0.9914
The F1 score is:  0.9772
The Matthews correlation coefficient is:  0.8441

Scaler:  RobustScaler 

The Accuracy is:  0.9575
The Precision is:  0.9611
The Recall is:  0.9898
The F1 score is:  0.9753
The Matthews correlation coefficient is:  0.8304

Scaler:  MinMaxScaler 

The Accuracy is:  0.9416
The Precision is:  0.9425
The Recall is:  0.9914
The F1 score is:  0.9663
The Matthews correlation coefficient is:  0.7612



Tendo em conta os resultados para o Logistic Regression temos que o **Standart Scaler** obteve melhores resultados

#### Decision Trees:

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,DecisionTreeClassifier(),results,X_train,X_test,y_train,y_test);




Scaler:  StandardScaler 

The Accuracy is:  0.9662
The Precision is:  0.9789
The Recall is:  0.9812
The F1 score is:  0.9800
The Matthews correlation coefficient is:  0.8699

Scaler:  RobustScaler 

The Accuracy is:  0.9575
The Precision is:  0.9741
The Recall is:  0.9757
The F1 score is:  0.9749
The Matthews correlation coefficient is:  0.8370

Scaler:  MinMaxScaler 

The Accuracy is:  0.9542
The Precision is:  0.9733
The Recall is:  0.9725
The F1 score is:  0.9729
The Matthews correlation coefficient is:  0.8252



Tendo em conta os resultados para as Decision Trees temos que o **Robust Scaler** obteve melhores resultados

#### Random Forests:

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,RandomForestClassifier(),results,X_train,X_test,y_train,y_test);

Scaler:  StandardScaler 

The Accuracy is:  0.9728
The Precision is:  0.9813
The Recall is:  0.9867
The F1 score is:  0.9840
The Matthews correlation coefficient is:  0.8948

Scaler:  RobustScaler 

The Accuracy is:  0.9735
The Precision is:  0.9805
The Recall is:  0.9882
The F1 score is:  0.9844
The Matthews correlation coefficient is:  0.8970

Scaler:  MinMaxScaler 

The Accuracy is:  0.9715
The Precision is:  0.9782
The Recall is:  0.9882
The F1 score is:  0.9832
The Matthews correlation coefficient is:  0.8888



Tendo em conta os resultados para o Random Forests temos que o **MinMaxScaler** obteve melhores resultados

####

#### SVM:

In [32]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

# Faz-se a divisao entre teste e treino 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}

testScalers(scalers,SVC(),results,X_train,X_test,y_train,y_test);

Scaler:  StandardScaler 

The Accuracy is:  0.9688
The Precision is:  0.9716
The Recall is:  0.9922
The F1 score is:  0.9817
The Matthews correlation coefficient is:  0.8770

Scaler:  RobustScaler 

The Accuracy is:  0.9608
The Precision is:  0.9620
The Recall is:  0.9929
The F1 score is:  0.9772
The Matthews correlation coefficient is:  0.8440

Scaler:  MinMaxScaler 

The Accuracy is:  0.9502
The Precision is:  0.9531
The Recall is:  0.9898
The F1 score is:  0.9711
The Matthews correlation coefficient is:  0.7992



Tendo em conta os resultados para o SVM temos que o **Standart Scaler** obteve melhores resultados

#### Tendo em conta que em todos os testes usando 4 modelos diferentes, o StandartScaler obteve os melhores resultados em 2 dos 4 testes, *a não ser que seja utilizado no final o modelo Decision Trees, ou Random Forests mas mesmo com estes modelos os valores do StandartScaler continuam bem positivos*, deve ser utilizado o método de scaling : **StandartScaler**

## Identificar e remover colunas irrelevantes

#### Usar o Correlation selection method para remover variáveis irrelavantes:

Para encontrarmos as variaveis irrelevantes temos de procurar as variaveis que têm uma correlação perto de 0 pois perto de 1 significa que apoiam fortemente a feature alvo (Biodegradable) e perto de -1 rejeitam fortemente a feature alvo, daí as que têm uma correlacao com valor perto de 0 serem irrelevantes pois nem apoiam nem rejeitam a feature alvo nao trazendo assim vantagens suficientes para continuarem no dataSet.
 

In [33]:
from sklearn import tree
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, confusion_matrix,matthews_corrcoef, precision_score, recall_score
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [34]:
# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

#Escala-se a data a ser utilizada, com o StandartScaler (melhor dos testes)
Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

results = {}
# Obter as stats antes da remocao das irrelevantes
print("Teste com * TODAS * as variáveis:")
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,X_train,X_test,y_train,y_test);



#REMOVER VARIÀVEIS IRRELEVANTES:

#print(pd.DataFrame(Xtrain_scaled))
N,M=Xtrain_scaled.shape
N,M
v=np.hstack((y_train.values.reshape((N,1)), Xtrain_scaled))
#print(pd.DataFrame(v))
corr_data = pd.DataFrame(np.corrcoef(v.T))
first_column = corr_data.iloc[:, 0]
#print(first_column);
positive = True
counter=1;

selColIndexs = []

while positive and counter<len(first_column):
    
    value = first_column.iloc[counter];

    if(value>0 or value<-0.35):
        selColIndexs.append(counter-1);
    counter+=1
        
# indices (já sem a coluna adicionada da variavel alvo) com as features que nao foram removidas - ou seja as relevantes;
selColIndexs.append(dataSet.shape[1]-1);

#Apartir de agora este é o novo DataSet contendo apenas as colunas relevantes
smallDataSet = dataSet.iloc[:,selColIndexs];

# Temos que X sao as features e o Y é a variavel de classificação
X = smallDataSet.drop('Biodegradable_RB', axis=1)
y = smallDataSet['Biodegradable_RB']

#Escala-se a data a ser utilizada, com o StandartScaler (melhor dos testes)
Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

#Numero de variaveis removidas de maneira a reduzir a dimensionalidade
print("Foram removidas",len(first_column) - len(selColIndexs),"features de modo a diminuir a dimensionalidade dos dados");


print("\n\nTeste após * SEREM REMOVIDAS AS VARIAVEIS IRRELEVANTES *");
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,X_train,X_test,y_train,y_test);

smallDataSet;

Teste com * TODAS * as variáveis:
Scaler:  StandartScaler 

The Accuracy is:  0.9728
The Precision is:  0.9798
The Recall is:  0.9882
The F1 score is:  0.9840
The Matthews correlation coefficient is:  0.8942

Foram removidas 22 features de modo a diminuir a dimensionalidade dos dados


Teste após * SEREM REMOVIDAS AS VARIAVEIS IRRELEVANTES *
Scaler:  StandartScaler 

The Accuracy is:  0.9701
The Precision is:  0.9738
The Recall is:  0.9914
The F1 score is:  0.9825
The Matthews correlation coefficient is:  0.8826



Desta maneira acabámos por reduzir a dimensionalidade dos dados, eliminando cerca de 22 colunas irrelevantes e ainda acabámos por melhorar um pouco a accuracy de previsão usando uma RandomForest e escalando os dados com StandartScaler



# | --------------------------------------------------- |
# | ---- EM PROGRESSO : A PARTIR DE AQUI ------|
# V -------------------------------------------------- V

#### Using Stepwise Feature selection method :


In [35]:
#ESTE PASSO COSTUMA DEMORAR CERCA DE 2 MIN A EXECUTAR !!

# Vamos tentar descobrir quais as 5 variáveis mais relevantes usando o Stepwise Feature selection.

from sklearn.feature_selection import SequentialFeatureSelector

# Temos que X sao as features e o Y é a variavel de classificação
X = smallDataSet.drop('Biodegradable_RB', axis=1)
y =smallDataSet['Biodegradable_RB']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)

N,M=Xtrain_scaled.shape

#using RandomForest for sequential feature selection
clf = RandomForestClassifier()
sfs = SequentialFeatureSelector(clf, n_features_to_select=5)
sfs.fit(Xtrain_scaled, y_train)

#get the relevant columns
features=sfs.get_support()
Features_selected =np.arange(M)[features]
print("The features selected are columns: ", Features_selected)




The features selected are columns:  [ 1  3  5  7 10]


In [36]:
print("Teste com * TODAS * as variáveis - utilizando o StandartScaler:")
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,X_train,X_test,y_train,y_test);

newXtrain =sfs.transform(Xtrain_scaled)
newXtest = sfs.transform(Xtest_scaled)


print("\n\n Teste com * APENAS AS VARIÀVEIS SELECIONADAS *  - utilizando o StandartScaler:");
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,newXtrain,newXtest,y_train,y_test);

Teste com * TODAS * as variáveis - utilizando o StandartScaler:
Scaler:  StandartScaler 

The Accuracy is:  0.9728
The Precision is:  0.9761
The Recall is:  0.9922
The F1 score is:  0.9840
The Matthews correlation coefficient is:  0.8933



 Teste com * APENAS AS VARIÀVEIS SELECIONADAS *  - utilizando o StandartScaler:
Scaler:  StandartScaler 

The Accuracy is:  0.9482
The Precision is:  0.9544
The Recall is:  0.9859
The F1 score is:  0.9699
The Matthews correlation coefficient is:  0.7915



#### Using PCA selection method :

In [37]:
import numpy as np
from sklearn.decomposition import PCA

# Temos que X sao as features e o Y é a variavel de classificação
X = dataSet.drop('Biodegradable_RB', axis=1)
y = dataSet['Biodegradable_RB']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)

print("Teste com * TODAS * as variáveis - utilizando o StandartScaler:")
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,Xtrain_scaled,Xtest_scaled,y_train,y_test);

#utilizamos 20 PCs pois após várias tentaivas é o valor que nos permite obter uma Total variance de pelo menos 90% 
pca = PCA(n_components=20) 
pca.fit(Xtrain_scaled)
tve=0 #total variance explained
for i, ve in enumerate(pca.explained_variance_ratio_):
    tve+=ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve) )
print()
#print("Actual Eigenvalues:", pca.singular_values_)

newX_train=pca.transform(Xtrain_scaled)
newX_test=pca.transform(Xtest_scaled)

print(newX_test.shape);

print("\n\n Teste com * APENAS AS VARIÀVEIS SELECIONADAS *  - utilizando o StandartScaler:");
testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,newX_train,newX_test,y_train,y_test);


Teste com * TODAS * as variáveis - utilizando o StandartScaler:
Scaler:  StandartScaler 

The Accuracy is:  0.9735
The Precision is:  0.9813
The Recall is:  0.9874
The F1 score is:  0.9844
The Matthews correlation coefficient is:  0.8972

PC0 - Variance explained:  0.1943 - Total Variance:  0.1943
PC1 - Variance explained:  0.1135 - Total Variance:  0.3078
PC2 - Variance explained:  0.1019 - Total Variance:  0.4097
PC3 - Variance explained:  0.0822 - Total Variance:  0.4920
PC4 - Variance explained:  0.0700 - Total Variance:  0.5619
PC5 - Variance explained:  0.0504 - Total Variance:  0.6123
PC6 - Variance explained:  0.0351 - Total Variance:  0.6474
PC7 - Variance explained:  0.0315 - Total Variance:  0.6789
PC8 - Variance explained:  0.0305 - Total Variance:  0.7094
PC9 - Variance explained:  0.0284 - Total Variance:  0.7378
PC10 - Variance explained:  0.0261 - Total Variance:  0.7639
PC11 - Variance explained:  0.0248 - Total Variance:  0.7886
PC12 - Variance explained:  0.0237 - To

In [38]:
# # Temos que X sao as features e o Y é a variavel de classificação
# X = dataSet.drop('Biodegradable_RB', axis=1)
# y = dataSet['Biodegradable_RB']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

# Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)
# print("Teste com * TODAS * as variáveis - utilizando o StandartScaler:")
# testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,Xtrain_scaled,Xtest_scaled,y_train,y_test);


# X_pca = pca.fit_transform(Xtrain_scaled)

# loadings = pca.components_

# feature_count = {}

# # Esta parte do código foi obtida com ajuda da ferramenta ChatGPT
# for i, component in enumerate(loadings):
#     feature_indices = np.argsort(np.abs(component))[::-1] 
#     contributing_features = feature_indices[:10]  
#     for feature_index in contributing_features:
#         feature_name = X.columns[feature_index]
#         if feature_name in feature_count:
#             feature_count[feature_name] += 1
#         else:
#             feature_count[feature_name] = 1

# sorted_features = sorted(feature_count.items(), key=lambda x: x[1], reverse=True)[:20]


# selected_feature_names = [feature for feature, count in sorted_features]

# filtered_dataset = dataSet.loc[:, selected_feature_names]
# filtered_dataset

# # Temos que X sao as features e o Y é a variavel de classificação
# X = filtered_dataset
# y = dataSet['Biodegradable_RB']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22)

# Xtrain_scaled, Xtest_scaled = scaleData(StandardScaler(),X_train,X_test)


# print("\n\n Teste com * APENAS AS VARIÀVEIS SELECIONADAS *  - utilizando o StandartScaler:");
# testScalers({'StandartScaler':StandardScaler()},RandomForestClassifier(),results,newX_train,newX_test,y_train,y_test);

# print("Features selecionadas:",selected_feature_names);

    

Tendo em conta os resultados, os métodos que mais afetam a dimensionalidade são a Correlation e a SFS mas o que melhores resultados apresenta é o PCA com um accuracy: 0.9615 e um F1 score: 0.9775 face aos 0.9721 e 0.9890 